In [1]:
# Imports
import json
import requests
import time
import numpy as np
import pandas as pd
from halo import halo

### Collecting Rank Data

Here we will be gathering rank data. I'll mostly just dump the data and deal with sorting it out more later on. That does mean the file will be large

halo_class = halo('username_data.csv')

In [2]:
halo_class = halo('ranks.csv')

halo_class.data = pd.concat([halo_class.data,halo_sub.data],copy=False,ignore_index=True)
halo_class.data = halo_class.data.drop(['LastMatch'],axis=1)

In [3]:
halo_class.data = halo_class.data.astype(object)
halo_class.data

,usernames,1v1,2v2,3v3
0,A 0 N Eclipse,NaN,NaN,NaN
1,A 007 sebastian,NaN,NaN,NaN
2,A 1 Armed Ninja,NaN,NaN,NaN
3,A 12ft Dogman,NaN,NaN,NaN
4,A 15 reasons,NaN,NaN,NaN
...,...,...,...,...
238174,zzzCJANDREW,NaN,NaN,NaN
238175,zzzDoge,NaN,NaN,NaN
238176,zzzOMGLOLzzz,NaN,NaN,NaN
238177,zzziNzEpTiOnZzz,NaN,NaN,NaN


In [6]:
# The list I'll be using
names = halo_class.gamertags(nasort='yes')
length = len(names)
# for loop with steps of 6
for i in range(0,length,6):
    try:
        # Basically checking that we aren't at the end
        if i < (int(length-6)):
            out = halo_class.getRanks(names[i:i+6])
            print(int(i/6),i,length-i)
        else:
            out = halo_class.getRanks(names[i:(i+(length%6)-1)])
            print(int(i/6),i,length-i)
    except KeyboardInterrupt:
        break
    except json.JSONDecodeError:
        print(i)

0 0 208467
1 6 208461
2 12 208455
3 18 208449
4 24 208443
5 30 208437
6 36 208431
7 42 208425
8 48 208419
9 54 208413
10 60 208407
11 66 208401
12 72 208395
13 78 208389
14 84 208383
15 90 208377
16 96 208371
17 102 208365
18 108 208359
19 114 208353
20 120 208347
21 126 208341
22 132 208335
23 138 208329
24 144 208323
25 150 208317
26 156 208311
27 162 208305
28 168 208299
29 174 208293
30 180 208287
31 186 208281
32 192 208275
33 198 208269
34 204 208263
35 210 208257
36 216 208251
37 222 208245
38 228 208239
39 234 208233
40 240 208227
41 246 208221
42 252 208215
43 258 208209
44 264 208203
45 270 208197
46 276 208191
47 282 208185
48 288 208179
49 294 208173
50 300 208167
51 306 208161
52 312 208155
53 318 208149
54 324 208143
55 330 208137
56 336 208131
57 342 208125
58 348 208119
59 354 208113
60 360 208107
61 366 208101
62 372 208095
63 378 208089
64 384 208083
65 390 208077
66 396 208071
67 402 208065
68 408 208059
69 414 208053
70 420 208047
71 426 208041
72 432 208035
73 438 

531 3186 205281
532 3192 205275
533 3198 205269
534 3204 205263
535 3210 205257
536 3216 205251
537 3222 205245
538 3228 205239
539 3234 205233
540 3240 205227
541 3246 205221
542 3252 205215
543 3258 205209
544 3264 205203
545 3270 205197
546 3276 205191
547 3282 205185
548 3288 205179
549 3294 205173
550 3300 205167
551 3306 205161
552 3312 205155
553 3318 205149
554 3324 205143
555 3330 205137
556 3336 205131
557 3342 205125
558 3348 205119
559 3354 205113
560 3360 205107
561 3366 205101
562 3372 205095
563 3378 205089
564 3384 205083
565 3390 205077
566 3396 205071
567 3402 205065
568 3408 205059
569 3414 205053
570 3420 205047
571 3426 205041
572 3432 205035
573 3438 205029
574 3444 205023
575 3450 205017
576 3456 205011
577 3462 205005
578 3468 204999
579 3474 204993
580 3480 204987
581 3486 204981
582 3492 204975
583 3498 204969
584 3504 204963
585 3510 204957
586 3516 204951
587 3522 204945
588 3528 204939
589 3534 204933
590 3540 204927
591 3546 204921
592 3552 204915
593 3558

1041 6246 202221
1042 6252 202215
1043 6258 202209
1044 6264 202203
1045 6270 202197
1046 6276 202191
1047 6282 202185
1048 6288 202179
1049 6294 202173
1050 6300 202167
1051 6306 202161
1052 6312 202155
1053 6318 202149
1054 6324 202143
1055 6330 202137
1056 6336 202131
1057 6342 202125
1058 6348 202119
1059 6354 202113
1060 6360 202107
1061 6366 202101
1062 6372 202095
1063 6378 202089
1064 6384 202083
1065 6390 202077
1066 6396 202071
1067 6402 202065
1068 6408 202059
1069 6414 202053
1070 6420 202047
1071 6426 202041
1072 6432 202035
1073 6438 202029
1074 6444 202023
1075 6450 202017
1076 6456 202011
1077 6462 202005
1078 6468 201999
1079 6474 201993
1080 6480 201987
1081 6486 201981
1082 6492 201975
1083 6498 201969
1084 6504 201963
1085 6510 201957
1086 6516 201951
1087 6522 201945
1088 6528 201939
1089 6534 201933
1090 6540 201927
1091 6546 201921
1092 6552 201915
1093 6558 201909
1094 6564 201903
1095 6570 201897
1096 6576 201891
1097 6582 201885
1098 6588 201879
1099 6594 2018

1523 9138 199329
1524 9144 199323
1525 9150 199317
1526 9156 199311
1527 9162 199305
1528 9168 199299
1529 9174 199293
1530 9180 199287
1531 9186 199281
1532 9192 199275
1533 9198 199269
1534 9204 199263
1535 9210 199257
1536 9216 199251
1537 9222 199245
1538 9228 199239
1539 9234 199233
1540 9240 199227
1541 9246 199221
1542 9252 199215
1543 9258 199209
1544 9264 199203
1545 9270 199197
1546 9276 199191
1547 9282 199185
1548 9288 199179
1549 9294 199173
1550 9300 199167
1551 9306 199161
1552 9312 199155
1553 9318 199149
1554 9324 199143
1555 9330 199137
1556 9336 199131
1557 9342 199125
1558 9348 199119
1559 9354 199113
1560 9360 199107
1561 9366 199101
1562 9372 199095
1563 9378 199089
1564 9384 199083
1565 9390 199077
1566 9396 199071
1567 9402 199065
1568 9408 199059
1569 9414 199053
1570 9420 199047
1571 9426 199041
1572 9432 199035
1573 9438 199029
1574 9444 199023
1575 9450 199017
1576 9456 199011
1577 9462 199005
1578 9468 198999
1579 9474 198993
1580 9480 198987
1581 9486 1989

1987 11922 196545
1988 11928 196539
1989 11934 196533
1990 11940 196527
1991 11946 196521
1992 11952 196515
1993 11958 196509
1994 11964 196503
1995 11970 196497
1996 11976 196491
1997 11982 196485
1998 11988 196479
1999 11994 196473
2000 12000 196467
2001 12006 196461
2002 12012 196455
2003 12018 196449
2004 12024 196443
2005 12030 196437
2006 12036 196431
2007 12042 196425
2008 12048 196419
2009 12054 196413
2010 12060 196407
2011 12066 196401
2012 12072 196395
2013 12078 196389
2014 12084 196383
2015 12090 196377
2016 12096 196371
2017 12102 196365
2018 12108 196359
2019 12114 196353
2020 12120 196347
2021 12126 196341
2022 12132 196335
2023 12138 196329
2024 12144 196323
2025 12150 196317
2026 12156 196311
2027 12162 196305
2028 12168 196299
2029 12174 196293
2030 12180 196287
2031 12186 196281
2032 12192 196275
2033 12198 196269
2034 12204 196263
2035 12210 196257
2036 12216 196251
2037 12222 196245
2038 12228 196239
2039 12234 196233
2040 12240 196227
2041 12246 196221
2042 12252

...

32576 195456 13011
32577 195462 13005
32578 195468 12999
32579 195474 12993
32580 195480 12987
32581 195486 12981
32582 195492 12975
32583 195498 12969
32584 195504 12963
32585 195510 12957
32586 195516 12951
32587 195522 12945
32588 195528 12939
32589 195534 12933
32590 195540 12927
32591 195546 12921
32592 195552 12915
32593 195558 12909
32594 195564 12903
32595 195570 12897
32596 195576 12891
32597 195582 12885
32598 195588 12879
32599 195594 12873
32600 195600 12867
32601 195606 12861
32602 195612 12855
32603 195618 12849
32604 195624 12843
32605 195630 12837
32606 195636 12831
32607 195642 12825
32608 195648 12819
32609 195654 12813
32610 195660 12807
32611 195666 12801
32612 195672 12795
32613 195678 12789
32614 195684 12783
32615 195690 12777
32616 195696 12771
32617 195702 12765
32618 195708 12759
32619 195714 12753
32620 195720 12747
32621 195726 12741
32622 195732 12735
32623 195738 12729
32624 195744 12723
32625 195750 12717
32626 195756 12711
32627 195762 12705
32628 195768

33008 198048 10419
33009 198054 10413
33010 198060 10407
33011 198066 10401
33012 198072 10395
33013 198078 10389
33014 198084 10383
33015 198090 10377
33016 198096 10371
33017 198102 10365
33018 198108 10359
33019 198114 10353
33020 198120 10347
33021 198126 10341
33022 198132 10335
33023 198138 10329
33024 198144 10323
33025 198150 10317
33026 198156 10311
33027 198162 10305
33028 198168 10299
33029 198174 10293
33030 198180 10287
33031 198186 10281
33032 198192 10275
33033 198198 10269
33034 198204 10263
33035 198210 10257
33036 198216 10251
33037 198222 10245
33038 198228 10239
33039 198234 10233
33040 198240 10227
33041 198246 10221
33042 198252 10215
33043 198258 10209
33044 198264 10203
33045 198270 10197
33046 198276 10191
33047 198282 10185
33048 198288 10179
33049 198294 10173
33050 198300 10167
33051 198306 10161
33052 198312 10155
33053 198318 10149
33054 198324 10143
33055 198330 10137
33056 198336 10131
33057 198342 10125
33058 198348 10119
33059 198354 10113
33060 198360

33460 200760 7707
33461 200766 7701
33462 200772 7695
33463 200778 7689
33464 200784 7683
33465 200790 7677
33466 200796 7671
33467 200802 7665
33468 200808 7659
33469 200814 7653
33470 200820 7647
33471 200826 7641
33472 200832 7635
33473 200838 7629
33474 200844 7623
33475 200850 7617
33476 200856 7611
33477 200862 7605
33478 200868 7599
33479 200874 7593
33480 200880 7587
33481 200886 7581
33482 200892 7575
33483 200898 7569
33484 200904 7563
33485 200910 7557
33486 200916 7551
33487 200922 7545
33488 200928 7539
33489 200934 7533
33490 200940 7527
33491 200946 7521
33492 200952 7515
33493 200958 7509
33494 200964 7503
33495 200970 7497
33496 200976 7491
33497 200982 7485
33498 200988 7479
33499 200994 7473
33500 201000 7467
33501 201006 7461
33502 201012 7455
33503 201018 7449
33504 201024 7443
33505 201030 7437
33506 201036 7431
33507 201042 7425
33508 201048 7419
33509 201054 7413
33510 201060 7407
33511 201066 7401
33512 201072 7395
33513 201078 7389
33514 201084 7383
33515 2010

33916 203496 4971
33917 203502 4965
33918 203508 4959
33919 203514 4953
33920 203520 4947
33921 203526 4941
33922 203532 4935
33923 203538 4929
33924 203544 4923
33925 203550 4917
33926 203556 4911
33927 203562 4905
33928 203568 4899
33929 203574 4893
33930 203580 4887
33931 203586 4881
33932 203592 4875
33933 203598 4869
33934 203604 4863
33935 203610 4857
33936 203616 4851
33937 203622 4845
33938 203628 4839
33939 203634 4833
33940 203640 4827
33941 203646 4821
33942 203652 4815
33943 203658 4809
33944 203664 4803
33945 203670 4797
33946 203676 4791
33947 203682 4785
33948 203688 4779
33949 203694 4773
33950 203700 4767
33951 203706 4761
33952 203712 4755
33953 203718 4749
33954 203724 4743
33955 203730 4737
33956 203736 4731
33957 203742 4725
33958 203748 4719
33959 203754 4713
33960 203760 4707
33961 203766 4701
33962 203772 4695
33963 203778 4689
33964 203784 4683
33965 203790 4677
33966 203796 4671
33967 203802 4665
33968 203808 4659
33969 203814 4653
33970 203820 4647
33971 2038

34372 206232 2235
34373 206238 2229
34374 206244 2223
34375 206250 2217
34376 206256 2211
34377 206262 2205
34378 206268 2199
34379 206274 2193
34380 206280 2187
34381 206286 2181
34382 206292 2175
34383 206298 2169
34384 206304 2163
34385 206310 2157
34386 206316 2151
34387 206322 2145
34388 206328 2139
34389 206334 2133
34390 206340 2127
34391 206346 2121
34392 206352 2115
34393 206358 2109
34394 206364 2103
34395 206370 2097
34396 206376 2091
34397 206382 2085
34398 206388 2079
34399 206394 2073
34400 206400 2067
34401 206406 2061
34402 206412 2055
34403 206418 2049
34404 206424 2043
34405 206430 2037
34406 206436 2031
34407 206442 2025
34408 206448 2019
34409 206454 2013
34410 206460 2007
34411 206466 2001
34412 206472 1995
34413 206478 1989
34414 206484 1983
34415 206490 1977
34416 206496 1971
34417 206502 1965
34418 206508 1959
34419 206514 1953
34420 206520 1947
34421 206526 1941
34422 206532 1935
34423 206538 1929
34424 206544 1923
34425 206550 1917
34426 206556 1911
34427 2065

In [7]:
halo_class.createfile('ranks.csv')

True

In [ ]:
# This one dude
df = halo_class.data
df[df.isnull().any(axis=1)]

In [ ]:
halo_class.data